In [2]:
%matplotlib inline
import pandas as pd 
import numpy as np

import torch 
from torch.utils.data import DataLoader
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder

from modules.classifer_utils import ClassifierDataset, TrainingManager


device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")




Using cpu device


In [3]:
# Load the Titanic dataset.
titanic_train_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/train.csv")
titanic_test_csv_df = pd.read_csv("https://storage.googleapis.com/tf-datasets/titanic/eval.csv")

print(f'titanic_train_csv_df shape: {titanic_train_csv_df.shape}')
print(f'titanic_test_csv_df shape: {titanic_test_csv_df.shape}')

# concat test and train to get metadata for non numeric categories
csv_union_df = pd.concat([titanic_test_csv_df, titanic_train_csv_df])



titanic_train_csv_df shape: (627, 10)
titanic_test_csv_df shape: (264, 10)


In [4]:

label_column_name = "survived"
[train_encoded_df, test_encoded_df] = ClassifierDataset.onehot_encode_datafames([titanic_train_csv_df, titanic_test_csv_df])


train_ds = ClassifierDataset(train_encoded_df, label_column_name)
test_ds = ClassifierDataset(test_encoded_df, label_column_name)

batch_size = int(len(train_ds) / 10)

train_dataloader = DataLoader(train_ds, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_ds, shuffle=True)
print(f'{len(train_ds)} training records in with batch size {batch_size}, {len(test_ds)} records for test')

print(f'train has {train_ds.get_feature_count()} features')
print(f'test has {test_ds.get_feature_count()} features')




627 training records in with batch size 62, 264 records for test
train has 23 features
test has 23 features


In [5]:
DROPOUT_RATE_01 = .20

# TODO have a config for layers and wire them from it

class TitanicSurvivalNeuralNetwork(nn.Module):
    def __init__(self, num_feature_columns):
        super().__init__()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(num_feature_columns, 64),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(64, 64),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(64, 32),
            nn.GELU(),
            nn.Dropout(p=DROPOUT_RATE_01), 
            nn.Linear(32, 32),
            nn.GELU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.linear_relu_stack(x)


first_training_record, _ = train_ds[0]
num_features = first_training_record.shape[-1]
model = TitanicSurvivalNeuralNetwork( num_features)
print(model)

TitanicSurvivalNeuralNetwork(
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=23, out_features=64, bias=True)
    (1): GELU(approximate='none')
    (2): Dropout(p=0.2, inplace=False)
    (3): Linear(in_features=64, out_features=64, bias=True)
    (4): GELU(approximate='none')
    (5): Dropout(p=0.2, inplace=False)
    (6): Linear(in_features=64, out_features=32, bias=True)
    (7): GELU(approximate='none')
    (8): Dropout(p=0.2, inplace=False)
    (9): Linear(in_features=32, out_features=32, bias=True)
    (10): GELU(approximate='none')
    (11): Linear(in_features=32, out_features=1, bias=True)
    (12): Sigmoid()
  )
)


In [ ]:


SEED = 123
torch.manual_seed(SEED)


if torch.cuda.is_available():
    torch.cuda.manual_seed(SEED)
    torch.cuda.manual_seed_all(SEED)


num_features = train_ds.get_feature_count()
model = TitanicSurvivalNeuralNetwork( num_features )
model.to(device)

loss_fn   = nn.BCELoss()  # binary cross entropy
optimizer = optim.Adam(model.parameters(), lr=0.001)

num_epochs = 100

for epoch in range(num_epochs):
    epoch_correct_count, epoch_pred_count = 0, 0
    for X, y in train_dataloader:

        X = X.to(device)
        y = y.reshape(-1, 1).to(device)

        y_pred = model(X)
        loss = loss_fn(y_pred, y)

        y_pred_guess = torch.round(y_pred)
        batch_num_correct = (y == y_pred_guess).sum()
        epoch_correct_count += batch_num_correct
        epoch_pred_count += len(y)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    print(f"Epoch [{epoch+1}/{num_epochs}], {epoch_correct_count} of {epoch_pred_count} correct {(100*epoch_correct_count/epoch_pred_count):.1f} %")


Epoch [1/100], 318 of 627 correct 50.7 %
Epoch [2/100], 386 of 627 correct 61.6 %
Epoch [3/100], 385 of 627 correct 61.4 %
Epoch [4/100], 398 of 627 correct 63.5 %
Epoch [5/100], 445 of 627 correct 71.0 %
Epoch [6/100], 488 of 627 correct 77.8 %
Epoch [7/100], 499 of 627 correct 79.6 %
Epoch [8/100], 494 of 627 correct 78.8 %
Epoch [9/100], 505 of 627 correct 80.5 %
Epoch [10/100], 506 of 627 correct 80.7 %
Epoch [11/100], 511 of 627 correct 81.5 %
Epoch [12/100], 515 of 627 correct 82.1 %
Epoch [13/100], 514 of 627 correct 82.0 %
Epoch [14/100], 514 of 627 correct 82.0 %
Epoch [15/100], 514 of 627 correct 82.0 %
Epoch [16/100], 520 of 627 correct 82.9 %
Epoch [17/100], 518 of 627 correct 82.6 %
Epoch [18/100], 514 of 627 correct 82.0 %
Epoch [19/100], 514 of 627 correct 82.0 %
Epoch [20/100], 517 of 627 correct 82.5 %
Epoch [21/100], 517 of 627 correct 82.5 %
Epoch [22/100], 523 of 627 correct 83.4 %
Epoch [23/100], 516 of 627 correct 82.3 %
Epoch [24/100], 517 of 627 correct 82.5 %
E

In [ ]:

training_mgr = TrainingManager()
training_mgr.eval(model, test_dataloader)
